In [1]:
% load_ext autoreload
% autoreload 2

import numpy as np
from helpers import make_data, display_world, display_world_2

In [2]:
num_landmarks      = 1        # number of landmarks
N                  = 2       # time steps
world_size         = 20.0    # size of world (square)

# robot parameters
measurement_range  = 10.0     # range at which we can sense landmarks
motion_noise       = 0.0      # noise in robot motion
measurement_noise  = 0.0      # noise in the measurements
distance           = 5.0     # distance by which robot (intends to) move each iteratation 

data = [[[[0, 2.0, 1.0]], [-4, 3]], [[[0, -2.0, 4.0]], []]]
# motion 10,10 =>6.00000,13.00000, lmark=8,9

In [3]:
def initialize_constraints_1(N, num_landmarks, world_size):
    n = N + num_landmarks
    new_omega = np.zeros((n, n))
    new_omega[0][0] = 1
    ## TODO: Define the constraint *vector*, xi
    ## you can assume that the robot starts out in the middle of the world with 100% confidence
    new_xi = np.zeros((n, 1))
    new_xi[0] = world_size/2
    return new_omega, new_xi   

In [18]:
def update_motion(midx, delta, omega, xi, motion_noise):
    conf = 1.0/motion_noise if motion_noise else 1.0
    omega[midx][midx] += conf      # omega for motion x
    omega[midx][midx+1] += -conf
    omega[midx+1][midx] += -conf
    omega[midx+1][midx+1] += conf          # omega for motion x

    xi[midx] -= delta    #
    xi[midx+1] += delta

def update_landmark(midx, lidx, delta, omega, xi, measurement_noise):
    conf = 1.0/measurement_noise if measurement_noise else 1.0
    print('update_landmark', midx, lidx, delta,conf)
    omega[midx][midx] += conf
    omega[midx][lidx] += -conf
    omega[lidx][midx] += -conf
    omega[lidx][lidx] += conf
    
    xi[midx] -= delta              # ? + measurement_noise??
    xi[lidx] += delta              #
    
def mu_from_positions(omega, xi, move_lm, motion_noise=0.0, measurement_noise=0.0):
    n_disp = len(move_lm)
    for i, m in enumerate(move_lm):
        if m[0]:
            update_motion(i, m[0], omega, xi, motion_noise)
        print('mv', m[0], xi)
        for lm in m[1]:
            update_landmark(i, lm[0]+n_disp, lm[1], omega, xi, measurement_noise)
            print('lm',i, lm[1], xi)
            
    print(omega, xi)
    omega_inv = np.linalg.inv(np.matrix(omega))
    mu = omega_inv*xi
    return mu


In [19]:
n = 4      # 3 pos and 1 lm
omega_t = np.zeros((n, n))
omega_t[0][0] = 1
xi_t = np.zeros((n, 1))
xi_t[0] = -3
mv_lm = [[5, [(0,10)]], [3, [(0,5)]], [None, [(0, 2)]]]
mu_from_positions(omega_t, xi_t, mv_lm)  # see '2_2. Include Landmarks, solution.ipynb', (-3, 5, 3, 10, 5, 2)

mv 5 [[-8.]
 [ 5.]
 [ 0.]
 [ 0.]]
update_landmark 0 3 10 1.0
lm 0 10 [[-18.]
 [  5.]
 [  0.]
 [ 10.]]
mv 3 [[-18.]
 [  2.]
 [  3.]
 [ 10.]]
update_landmark 1 3 5 1.0
lm 1 5 [[-18.]
 [ -3.]
 [  3.]
 [ 15.]]
mv None [[-18.]
 [ -3.]
 [  3.]
 [ 15.]]
update_landmark 2 3 2 1.0
lm 2 2 [[-18.]
 [ -3.]
 [  1.]
 [ 17.]]
[[ 3. -1.  0. -1.]
 [-1.  3. -1. -1.]
 [ 0. -1.  2. -1.]
 [-1. -1. -1.  3.]] [[-18.]
 [ -3.]
 [  1.]
 [ 17.]]


matrix([[-3.],
        [ 2.],
        [ 5.],
        [ 7.]])

In [63]:
def slam(data, N, num_landmarks, world_size, motion_noise, measurement_noise):
    x_omega, x_xi = initialize_constraints_1(N, num_landmarks, world_size)
    y_omega, y_xi = initialize_constraints_1(N, num_landmarks, world_size)
    x_list, y_list = [], []   # [move, [list_lm]] on an axis  
    for i in range(len(data)):
        x_move = data[i][1][0] if data[i][1] else None
        y_move = data[i][1][1] if data[i][1] else None
        lm_x, lm_y = [], []
        for m_lm in data[i][0]:
            lm_x.append((m_lm[0], m_lm[1]))
            lm_y.append((m_lm[0], m_lm[2]))
        x_list.append((x_move, lm_x))
        y_list.append((y_move, lm_y))
    print(x_list, y_list)
    mu_x = mu_from_positions(x_omega, x_xi, x_list)
    mu_y = mu_from_positions(y_omega, y_xi, y_list)
    mu_x_a = np.squeeze(np.asarray(mu_x))
    mu_y_a = np.squeeze(np.asarray(mu_y))
#     print('mu_x:', type(mu_x_a), mu_x_a, '\nmu_y:',  mu_y_a.shape,mu_y_a)
#     return list(zip(mu_x_a.tolist(), mu_y_a.tolist()))
    return np.array([[mu_x_a[i], mu_y_a[i]] for i in range(len(mu_x_a))])


In [64]:
data = [[[[0, 2.0, 1.0]], [-4, 3]], [[[0, -2.0, 4.0]], []]]
slam(data, N, 1, 20, 0.0, 0.0)

[(-4, [(0, 2.0)]), (None, [(0, -2.0)])] [(3, [(0, 1.0)]), (None, [(0, 4.0)])]
mv -4 [[ 14.]
 [ -4.]
 [  0.]]
update_landmark 0 2 2.0 1.0
lm 0 2.0 [[ 12.]
 [ -4.]
 [  2.]]
mv None [[ 12.]
 [ -4.]
 [  2.]]
update_landmark 1 2 -2.0 1.0
lm 1 -2.0 [[ 12.]
 [ -2.]
 [  0.]]
[[ 3. -1. -1.]
 [-1.  2. -1.]
 [-1. -1.  2.]] [[ 12.]
 [ -2.]
 [  0.]]
mv 3 [[ 7.]
 [ 3.]
 [ 0.]]
update_landmark 0 2 1.0 1.0
lm 0 1.0 [[ 6.]
 [ 3.]
 [ 1.]]
mv None [[ 6.]
 [ 3.]
 [ 1.]]
update_landmark 1 2 4.0 1.0
lm 1 4.0 [[ 6.]
 [-1.]
 [ 5.]]
[[ 3. -1. -1.]
 [-1.  2. -1.]
 [-1. -1.  2.]] [[ 6.]
 [-1.]
 [ 5.]]


array([[ 10.        ,  10.        ],
       [  8.66666667,  11.        ],
       [  9.33333333,  13.        ]])